In [1]:
import os
from novel_risk_assistant_only_select_choice import (
    get_risk_th_list,
    get_customer_data_pre_mapping_list,
    get_customer_data_to_risk_th_list,
    get_risk_th_to_risk_desc_th_dict,
    get_risk_th_to_risk_name_and_desc_th_dict,
    get_risk_desc_th_list,
    get_risk_name_and_desc_th_list,
    get_customer_risk_name_to_risk_desc_dict,
    get_risk_3_type_to_risk_cat_dict,
    get_customer_risk_3_type_to_risk_cat_dict,
)
import json

In [2]:
dir_path = os.getcwd()
risk_data_df_path = os.path.join(dir_path, "data/20240811_ALL_risks.xlsx")
all_risk_th_list = get_risk_th_list(risk_data_df_path)
all_risk_desc_th_list = get_risk_desc_th_list(risk_data_df_path)
all_risk_name_and_desc_th_list = get_risk_name_and_desc_th_list(risk_data_df_path)

print(f"{len(all_risk_th_list)=}")
customer_data_df_path = os.path.join(
    dir_path, "data/customquestion_mapping_done_update risk detail.xlsx"
)
# create a interest df
customer_data_pre_mapping_list, customer_data_description_pre_mapping_list = (
    get_customer_data_pre_mapping_list(customer_data_df_path)
)
customer_risk_name_to_risk_desc_dict = get_customer_risk_name_to_risk_desc_dict(
    customer_data_df_path
)
# save customer_data_pre_mapping_list for debugging purposes
debug_file_path = os.path.join(dir_path, "data/debug.json")
with open(debug_file_path, "w") as f:
    json.dump(customer_data_pre_mapping_list, f, ensure_ascii=False)

customer_data_to_risk_th_list = get_customer_data_to_risk_th_list(
    customer_data_pre_mapping_list, all_risk_th_list
)
risk_th_to_risk_desc_th_dict = get_risk_th_to_risk_desc_th_dict(risk_data_df_path)
risk_th_to_risk_name_and_desc_th_dict = get_risk_th_to_risk_name_and_desc_th_dict(
    risk_data_df_path
)

len(all_risk_th_list)=165
before 158
after 157


In [3]:
len(all_risk_th_list), len(set(all_risk_th_list))

(165, 165)

In [4]:
customer_data_to_risk_th_list[:5]

[['เครื่องจักร Brakedown บ่อย', 'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย'],
 ['เครื่องจักรมีการ Break Down กระทบประสิทธิภาพการผลิตสินค้า',
  'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย'],
 ['ความสูญเสียจากความผิดพลาดจากการผลิต',
  'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย'],
 ['ระบบชลประทาน แบบ สปริงเกลอร์ ขนาดใหญ่',
  'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย'],
 ['ภาวะ EL NINO', 'เหตุการณ์สภาพภูมิอากาศสุดขั้ว']]

In [5]:
import itertools

# risk_th_to_risk_desc_th_dict
# risk_th_to_risk_name_and_desc_th_dict
print(
    *(
        f"{k}: {v}"
        for k, v in itertools.islice(risk_th_to_risk_desc_th_dict.items(), 3)
    ),
    sep="\n",
)

การขาดแคลนวัตถุดิบ : การขาดแคลนหรือข้อบกพร่องในวัตถุดิบหรือส่วนประกอบที่จําเป็นสําหรับสินค้าหรือบริการของบริษัท ยกเว้นปัญหาที่เกี่ยวข้องกับแรงงาน
วัตถุดิบไม่ได้คุณภาพ: ความเสี่ยงที่ซัพพลายเออร์จะไม่สามารถส่งมอบสินค้าหรือวัตถุดิบที่ตรงตามมาตรฐานคุณภาพที่กําหนด ซึ่งนําไปสู่ความล่าช้า การทํางานซ้ำหรือข้อบกพร่องของสินค้าหรือบริการ
อำนาจต่อรองกับซัพพลายเออร์ต่ำ: อำนาจต่อรองกับซัพพลายเออร์ต่ำสามารถส่งผลให้เกิดข้อเสียเปรียบด้านการซื้อขาย เช่น ต้นทุนที่สูงขึ้น คุณภาพที่ลดลง และความยืดหยุ่นที่น้อยลงในแง่ของเงื่อนไขการส่งมอบและการบริการ


In [6]:
from compare_models import (
    compare_target_sentence_rankings,
    cosine_distance,
    euclidean_distance,
)
import pandas as pd
from sentence_transformers import SentenceTransformer
from embedding_providers import (
    OpenAIEmbeddingProvider,
    SentenceTransformerProvider,
    GeminiEmbeddingProvider,
)
import openai
from dotenv import load_dotenv
import os

env_path = "/Users/ford/Documents/coding/confidential/.env"
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "API key is missing"
# Set your OpenAI API key
openai.api_key = api_key

# Initialize embedding providers with caching
embedding_models = {
    "all-MiniLM-L6-v2": SentenceTransformerProvider("all-MiniLM-L6-v2"),
    # "all-mpnet-base-v2": SentenceTransformerProvider("all-mpnet-base-v2"),
    "openai-small": OpenAIEmbeddingProvider(model_name="text-embedding-3-small"),
    "openai-large": OpenAIEmbeddingProvider(model_name="text-embedding-3-large"),
    "gemini-exp-03-07": GeminiEmbeddingProvider(
        model_name="gemini-embedding-exp-03-07"
    ),
}

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
candidate_sentences_risk_name = all_risk_th_list
target_sentences_risk_name = [i[1] for i in customer_data_to_risk_th_list]

candidate_sentences_risk_desc = all_risk_desc_th_list
target_sentences_risk_desc = [
    risk_th_to_risk_desc_th_dict[risk[1]] for risk in customer_data_to_risk_th_list
]

candidate_sentences_risk_name_and_desc = all_risk_name_and_desc_th_list
target_sentences_risk_name_and_desc = [
    risk_th_to_risk_name_and_desc_th_dict[risk[1]]
    for risk in customer_data_to_risk_th_list
]

In [8]:
# # risk_name
# input_sentences = [risk[0] for risk in customer_data_to_risk_th_list]

# risk_desc
# input_sentences_risk_name = [risk[0] for risk in customer_data_to_risk_th_list]
# input_sentences = [
#     customer_risk_name_to_risk_desc_dict[risk_name]
#     for risk_name in input_sentences_risk_name
# ]

# risk_name_and_desc
input_sentences_risk_name = [risk[0] for risk in customer_data_to_risk_th_list]
input_sentences_risk_desc = [
    customer_risk_name_to_risk_desc_dict[risk_name]
    for risk_name in input_sentences_risk_name
]
input_sentences = [
    risk_name + "\n" + risk_desc
    for risk_name, risk_desc in zip(
        input_sentences_risk_name, input_sentences_risk_desc
    )
]
input_sentences_map_to_risk_name = {
    input_sentence: risk_name
    for input_sentence, risk_name in zip(input_sentences, input_sentences_risk_name)
}


candidate_sets = {
    "candidate_set_risk_name": {
        "candidate_sentences": candidate_sentences_risk_name,
        "target_sentences": target_sentences_risk_name,
    },
    "candidate_set_risk_desc": {
        "candidate_sentences": candidate_sentences_risk_desc,
        "target_sentences": target_sentences_risk_desc,
    },
    "candidate_set_risk_name_and_desc": {
        "candidate_sentences": candidate_sentences_risk_name_and_desc,
        "target_sentences": target_sentences_risk_name_and_desc,
    },
}


# Initialize distance functions
distance_functions = {"cosine": cosine_distance, "euclidean": euclidean_distance}

comparison_results = compare_target_sentence_rankings(
    input_sentences, candidate_sets, embedding_models, distance_functions
)
comparison_result_df = pd.DataFrame(comparison_results)

Processing comparisons:   0%|          | 1/2568 [00:00<09:03,  4.72it/s]

Processing comparisons: 100%|██████████| 2568/2568 [08:47<00:00,  4.87it/s]


In [9]:
len(input_sentences)

107

In [10]:
candidate_sets

{'candidate_set_risk_name': {'candidate_sentences': ['การขาดแคลนวัตถุดิบ ',
   'วัตถุดิบไม่ได้คุณภาพ',
   'อำนาจต่อรองกับซัพพลายเออร์ต่ำ',
   'การขาดความร่วมมือภายในองค์กร',
   'กระบวนการทำงานไม่มีประสิทธิภาพ',
   'การขาดประสิทธิภาพในการจัดจ้างงานภายนอก (outsourcing)',
   'ข้อผิดพลาดทางบัญชี',
   'ความล้มเหลวในการดําเนินธุรกิจใหม่',
   'เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย',
   'เครื่องจักรหรืออุปกรณ์สูญหาย',
   'ยานพาหนะขัดข้องหรือเสียหาย',
   'ยานพาหนะสูญหาย',
   'โครงสร้างพื้นฐานขัดข้องหรือเสียหาย',
   'ผลิตภัณฑ์ไม่ได้คุณภาพ',
   'ความไม่ปลอดภัยของผลิตภัณฑ์',
   'การให้บริการไม่ได้คุณภาพ',
   'สินค้าคงคลังน้อยเกินไป',
   'สินค้าคงคลังมากเกินไป',
   'สินค้าคงคลังล้าสมัย',
   'สินค้าคงคลังหมดอายุ',
   'สินค้าคงคลังเสียหาย',
   'สินค้าคงคลังสูญหาย',
   'การจัดส่งสินค้าผิด',
   'สินค้าเสียหายระหว่างการจัดส่ง',
   'สินค้าสูญหายระหว่างการจัดส่ง',
   'ไม่สามารถจัดส่งสินค้าได้',
   'การจัดส่งสินค้าล่าช้า',
   'ผลิตภัณฑ์ไม่สามารถแข่งขันได้',
   'การให้บริการที่ไม่สามารถแข่งขันได้',
   'ก

In [11]:
comparison_result_df.head()

,input_sentence,target_sentence,candidate_sentences,embedded_model,distance_method,sorted_similar_sentences,sorted_similar_sentences_indices,target_order_in_sorted_similar_sentences
0,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",all-MiniLM-L6-v2,cosine,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119
1,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",all-MiniLM-L6-v2,euclidean,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119
2,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-small,cosine,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1
3,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-small,euclidean,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1
4,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-large,cosine,"[เครื่องจักรหรืออุปกรณ์สูญหาย, เครื่องจักรหรือ...","[9, 8, 4, 60, 57, 15, 61, 48, 67, 76, 13, 43, ...",2


In [12]:
risk_3_type_to_risk_cat_dict = get_risk_3_type_to_risk_cat_dict(risk_data_df_path)
# TODO map risk_name_to_risk_cat_dict


customer_risk_3_type_to_risk_cat_dict = get_customer_risk_3_type_to_risk_cat_dict(
    customer_data_df_path
)


# comparison_result_df add new column risk_name from input_sentences_map_to_risk_name
# comparison_result_df["risk_name"] = comparison_result_df["input_sentence"].map(
#     customer_risk_name_to_risk_cat_dict
# )
# comparison_result_df add new column customer_risk_cat from customer_risk_name_to_risk_cat_dict
# comparison_result_df["customer_risk_cat"] = comparison_result_df["input_sentence"].map(
#     customer_risk_3_type_to_risk_cat_dict
# )
def map_with_error_check(x):
    if x not in customer_risk_3_type_to_risk_cat_dict:
        raise KeyError(f"Key '{x}' not found in customer_risk_3_type_to_risk_cat_dict")
    return customer_risk_3_type_to_risk_cat_dict[x]


comparison_result_df["customer_risk_cat"] = comparison_result_df[
    "input_sentence"
].apply(map_with_error_check)
comparison_result_df["target_risk_cat"] = comparison_result_df["target_sentence"].map(
    risk_3_type_to_risk_cat_dict
)
# for candidate_sentences sorted_similar_sentences and sorted_similar_sentences_indices apply json.dumps
# comparison_result_df["candidate_sentences"] = comparison_result_df[
#     "candidate_sentences"
# ].apply(json.dumps)
# comparison_result_df["sorted_similar_sentences"] = comparison_result_df[
#     "sorted_similar_sentences"
# ].apply(json.dumps)
# comparison_result_df["sorted_similar_sentences_indices"] = comparison_result_df[
#     "sorted_similar_sentences_indices"
# ].apply(json.dumps)

In [13]:
comparison_result_df.head()

,input_sentence,target_sentence,candidate_sentences,embedded_model,distance_method,sorted_similar_sentences,sorted_similar_sentences_indices,target_order_in_sorted_similar_sentences,customer_risk_cat,target_risk_cat
0,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",all-MiniLM-L6-v2,cosine,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk
1,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",all-MiniLM-L6-v2,euclidean,[ด้านความปลอดภัยและอาชีวอนามัยของพนักงาน ลูกค้...,"[50, 63, 147, 153, 127, 128, 129, 130, 131, 15...",119,Operational risk,Operational risk
2,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-small,cosine,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk
3,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-small,euclidean,"[เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย, เคร...","[8, 9, 57, 20, 95, 98, 27, 13, 39, 138, 23, 76...",1,Operational risk,Operational risk
4,เครื่องจักร Brakedown บ่อย\nเครื่องจักร Braked...,เครื่องจักรหรืออุปกรณ์ขัดข้องหรือเสียหาย,"[การขาดแคลนวัตถุดิบ , วัตถุดิบไม่ได้คุณภาพ, อำ...",openai-large,cosine,"[เครื่องจักรหรืออุปกรณ์สูญหาย, เครื่องจักรหรือ...","[9, 8, 4, 60, 57, 15, 61, 48, 67, 76, 13, 43, ...",2,Operational risk,Operational risk


In [14]:
# 18 >>> 3 set 3 model 2 distance

In [15]:
# save comparison_result_df to excel file

comparison_result_df.to_excel("comparison_result_df.xlsx")

In [16]:
# save to pickle
import pickle

# save comparison_result_df to pickle
with open("comparison_result_df.pkl", "wb") as f:
    pickle.dump(comparison_result_df, f)

In [17]:
comparison_result_df.shape

(2568, 10)